# Project 1: Digital Divide
### Data Analysis

#### Based on PPIC's Just the Facts report ["California's Digital Divide"](https://www.ppic.org/publication/californias-digital-divide/)

## Research Question(s):
1. What share households with school-age children in X state have access to high-speed internet? 
2. Does this number vary across demographic groups? (in this case race/ethnicity).

## Goal:
* Use our `working-data dataset` (created in [Data_Prep notebook](00_DigitalDivide_Data_Prep.ipynb) notebook) to answer our research questions.

## Context:
* I have run through the data prep process in `00_DigitalDivide_Data_Prep` notebook. The resulting dataframe __`working_data_2_2019-05-01.dta`__ includes georgia households as described in 1970 and 1990, but not 2000 due to the small data set from 2000. It is also limited to households with K-12 aged children (ages 5-18).

***

#### Step 1: Set up your working environment.

Import all necessary libraries and create `Path`s to your data directories. This ensures reproducibility across file systems (windows uses `\` instead of `/`)

We need 
1. `pandas` to work with the data.
2. `pathlib`, and more specifically its `Path` object, to work with paths. This will ensure our code works in both Windows (which uses `\` in its file paths) and MacOS/Linux (which uses `/`).
3. `datetime` - tip: There are version control systems for data but tagging your data files with the date is not a bad first step if you're getting started.
4. `tree` - to display a directory's tree.

In [1]:
# setting up working environment
import pandas as pd
from pathlib import Path
from tools import tree
from datetime import datetime as dt
today = dt.today().strftime("%Y-%m-%d")

print(today)

2019-05-01


In [2]:
# data folder and paths
RAW_DATA_PATH = Path("../data/raw/")
INT_DATA_PATH = Path("../data/interim/")
PROC_DATA_PATH = Path("../data/processed/")
FIN_DATA_PATH = Path("../data/final/")

In [4]:
tree(INT_DATA_PATH)

+ ../data/interim
    + placeholder
    + state_data-01-May-19.dta
    + state_data_2019-05-01.dta
    + working_data-01-May-19.dta
    + working_data_2_2019-05-01.dta


In [8]:
data = pd.read_stata(INT_DATA_PATH / f'working_data_2_{today}.dta')

In [9]:
data.shape

(40737, 18)

In [10]:
data.head()

,year,serial,hhwt,statefip,countyfip,gq,cinethh,cihispeed,pernum,perwt,relate,related,sex,age,race,raced,hispan,hispand
0,2017,356883,178,georgia,0,households under 1970 definition,"yes, with a subscription to an internet service",no,1,178,head/householder,head/householder,female,37,white,white,not hispanic,not hispanic
1,2017,356883,178,georgia,0,households under 1970 definition,"yes, with a subscription to an internet service",no,2,210,spouse,spouse,male,47,white,white,not hispanic,not hispanic
2,2017,356883,178,georgia,0,households under 1970 definition,"yes, with a subscription to an internet service",no,3,153,child,child,male,14,white,white,not hispanic,not hispanic
3,2017,356883,178,georgia,0,households under 1970 definition,"yes, with a subscription to an internet service",no,4,157,child,child,female,8,white,white,not hispanic,not hispanic
4,2017,356887,83,georgia,0,households under 1970 definition,"yes, with a subscription to an internet service","yes (cable modem, fiber optic or dsl service)",1,83,head/householder,head/householder,male,49,white,white,not hispanic,not hispanic


In [11]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 40737 entries, 0 to 40736
Data columns (total 18 columns):
year         40737 non-null category
serial       40737 non-null int32
hhwt         40737 non-null int16
statefip     40737 non-null category
countyfip    40737 non-null int16
gq           40737 non-null category
cinethh      40737 non-null category
cihispeed    40737 non-null category
pernum       40737 non-null int8
perwt        40737 non-null int16
relate       40737 non-null category
related      40737 non-null category
sex          40737 non-null category
age          40737 non-null category
race         40737 non-null category
raced        40737 non-null category
hispan       40737 non-null category
hispand      40737 non-null category
dtypes: category(13), int16(3), int32(1), int8(1)
memory usage: 1.3 MB


Our **unit of observation** is still a (weighted) person but we're interested in **household-level** data. 

From IPUMS docs:
>HHWT indicates how many households in the U.S. population are represented by a given household in an IPUMS sample. <br><br>
>It is generally a good idea to use HHWT when conducting a household-level analysis of any IPUMS sample. The use of HHWT is optional when analyzing one of the "flat" or unweighted IPUMS samples. Flat IPUMS samples include the 1% samples from 1850-1930, all samples from 1960, 1970, and 1980, the 1% unweighted samples from 1990 and 2000, the 10% 2010 sample, and any of the full count 100% census datasets. HHWT must be used to obtain nationally representative statistics for household-level analyses of any sample other than those.<br><br>
>**Users should also be sure to select one person (e.g., PERNUM = 1) to represent the entire household.**

***

#### Step 2: Drop all observations were `pernum` doesn't equal 1

In [15]:
mask_pernum = (data['pernum'] == 1)

In [16]:
data[mask_pernum].shape

(10100, 18)

Save your data to an appropriately named variable.

In [26]:
state_households = data[mask_pernum].copy()

***

#### Step 3: Familiarize yourself with your variables of interest

From IPUMS [docs](https://usa.ipums.org/usa-action/variables/CINETHH#description_section):

>CINETHH reports whether any member of the household accesses the Internet. Here, "access" refers to whether or not someone in the household uses or connects to the Internet, regardless of whether or not they pay for the service.

In [20]:
# find the value_counts for your cinethh series
state_households['cinethh'].value_counts()

yes, with a subscription to an internet service                9238
no internet access at this house, apartment, or mobile home     643
yes, without a subscription to an internet service              219
Name: cinethh, dtype: int64

From IPUMS [docs](https://usa.ipums.org/usa-action/variables/CIHISPEED#description_section):
>CIHISPEED reports whether the respondent or any member of their household subscribed to the Internet using broadband (high speed) Internet service such as cable, fiber optic, or DSL service. <br><br>
>User Note: The ACS 2016 introduced changes to the questions regarding computer use and Internet access. See the comparability section and questionnaire text for more information. Additional information provided by the Census Bureau regarding these question alterations are available in the report: ACS Content Test Shows Need to Update Terminology

In [21]:
# find the value_counts for your cihispeed series
state_households['cihispeed'].value_counts()

yes (cable modem, fiber optic or dsl service)    7622
no                                               1616
n/a (gq)                                          862
Name: cihispeed, dtype: int64

_quick tip_ `.value_counts()` _has a_ `normalize` _parameter:_

In [13]:
pd.Series.value_counts?

Signature:
pd.Series.value_counts(
    self,
    normalize=False,
    sort=True,
    ascending=False,
    bins=None,
    dropna=True,
)
Docstring:
Return a Series containing counts of unique values.

The resulting object will be in descending order so that the
first element is the most frequently-occurring element.
Excludes NA values by default.

Parameters
----------
normalize : boolean, default False
    If True then the object returned will contain the relative
    frequencies of the unique values.
sort : boolean, default True
    Sort by values.
ascending : boolean, default False
    Sort in ascending order.
bins : integer, optional
    Rather than count values, group them into half-open bins,
    a convenience for ``pd.cut``, only works with numeric data.
dropna : boolean, default True
    Don't include counts of NaN.

Returns
-------
counts : Series

See Also
--------
Series.count: Number of non-NA elements in a Series.
DataFrame.count: Number of non-NA elements in a DataFrame.



In [23]:
# try it on your cinethh series
state_households['cinethh'].value_counts(normalize=True)

yes, with a subscription to an internet service                0.914653
no internet access at this house, apartment, or mobile home    0.063663
yes, without a subscription to an internet service             0.021683
Name: cinethh, dtype: float64

In [24]:
# on cihispeed 
state_households['cihispeed'].value_counts(normalize=True)

yes (cable modem, fiber optic or dsl service)    0.754653
no                                               0.160000
n/a (gq)                                         0.085347
Name: cihispeed, dtype: float64

***

This would be the end of our analysis if we weren't working with **weighted** data. **Weighted** data means each of our observations represent more than one person or household.

`perwt` = "Person's weight"

`hhwt` = "Household's weight"

`.value_counts(normalize=True)` counts the number of **observations** for each of a series' values and then divides it by the total count. If each of our observations was 1 person/household, we would have the answer already. 

What we need to do is **aggregate**.

***

#### Step 4: Grouping and aggregating data

The mechanics are kind of the same: 
1. Count the number of observations each that match each of the values in a series.
2. Add up **not the number of observations** but the weight of each observation.
3. Divide by the total.

#### Step 4.1: Group your data by their corresponding values

In [25]:
state_households.groupby("_________")

From the [docs](http://pandas.pydata.org/pandas-docs/stable/user_guide/groupby.html):

>A groupby operation involves some combination of splitting the
object, __applying a function__, and combining the results. This can be
used to group large amounts of data and compute operations on these
groups.

We're missing the **applying a function** part of it.

Try the following:
```python
state_households.groupby("countyfip").sum()
```

you can pass _almost_ any function to this. 

Try `.mean()`, `.max()`, `.min()`, `.std()`.

In [27]:
state_households.groupby('countyfip').sum()

,serial,hhwt,pernum,perwt
countyfip,,,,
0,2088302141,554659.0,5547.0,554685.0
15,36373988,11459.0,97.0,11470.0
21,61747619,14114.0,164.0,14115.0
45,34525624,13380.0,92.0,13382.0
51,94665665,25475.0,252.0,25475.0
57,81118710,28670.0,216.0,28673.0
59,31209030,10303.0,83.0,10303.0
63,96206684,33612.0,256.0,33608.0
67,277187962,85098.0,737.0,85105.0


You can select columns just like you would any other regular dataframe.

In [28]:
state_households.groupby("cihispeed")['hhwt'].sum()

cihispeed
n/a (gq)                                         111422.0
yes (cable modem, fiber optic or dsl service)    794633.0
no                                               180050.0
Name: hhwt, dtype: float64

***

In [31]:
n_households = state_households.groupby("cihispeed")['hhwt'].sum()[1]
_state = state_households['statefip'].unique()[0]
print(f"""
We can see now {n_households:,} households in {_state} have access to high-speed internet. But, out of how many?

To make this easier to follow, let's save our results to a variable:
""")


We can see now 794,633.0 households in georgia have access to high-speed internet. But, out of how many?

To make this easier to follow, let's save our results to a variable:



In [34]:
households_with_highspeed_access = state_households.groupby("cihispeed")['hhwt'].sum()

households_with_highspeed_access

cihispeed
n/a (gq)                                         111422.0
yes (cable modem, fiber optic or dsl service)    794633.0
no                                               180050.0
Name: hhwt, dtype: float64

This looks like any regular `pandas.Series`, how do we find the total `.sum()` of a series elements?

![math](../../static/math.png)

In [36]:
total_households = households_with_highspeed_access.sum()

That's our denominator! 

![nice](../../static/nooice.gif)

***

When you _apply_ and operation to a `pandas.Series` it _maps_ to each of its elements.

Try the following:
```python
households_with_highspeed_access * 1_000_000
```

```python
households_with_highspeed_access + 1_000_000
```

```python
households_with_highspeed_access / 1_000_000
```

In [37]:
households_with_highspeed_access * 1_000_000

cihispeed
n/a (gq)                                         1.114220e+11
yes (cable modem, fiber optic or dsl service)    7.946330e+11
no                                               1.800500e+11
Name: hhwt, dtype: float64

In [38]:
households_with_highspeed_access + 1_000_000

cihispeed
n/a (gq)                                         1111422.0
yes (cable modem, fiber optic or dsl service)    1794633.0
no                                               1180050.0
Name: hhwt, dtype: float64

In [39]:
households_with_highspeed_access / 1_000_000

cihispeed
n/a (gq)                                         0.111422
yes (cable modem, fiber optic or dsl service)    0.794633
no                                               0.180050
Name: hhwt, dtype: float64

Now that you know the denominator of our equation (how many households total in X state), how would you find each of the 3 values in your `households_with_highspeed_access` share of the total?

In [40]:
households_with_highspeed_access / total_households

cihispeed
n/a (gq)                                         0.102589
yes (cable modem, fiber optic or dsl service)    0.731636
no                                               0.165776
Name: hhwt, dtype: float64

***
***

### Part 2 of analysis: Creating derived variables

Now that you have answered **Research Question 1**, we can move on to Q2: 
>_Does this number vary across demographic groups? (in this case race/ethnicity)._

pandas `.groupby()` function can take a list of columns by which to group by 

Try the following:
```python
state_households.groupby(['race', 'cihispeed'])[['hhwt']].sum()
```

_Notice that I'm passing_ `[['hhwt']]` _(a 1-element list) and not just_ `['hhwt']` _try both yourself and let's discuss what's the difference._

***

#### Step 1: Define your groups



Pandas' `.loc` indexer serves not only to slice dataframes but also to assign new values to certain slices of dataframes.

For example,
```python
mask_madeup_data = (data['column_1'] == 'no answer')
data.loc[mask_madeup_data, 'new_column'] = 'this row did not answer'
```

The code above grabs all the rows that satisfy the condition and then looks at `'new_column'`, if it doesn't exist, it'll create it for you and assign the value `'this row did not answer'` to all the rows that match the condition. The rest will be filled with null values (NaNs).

###### Let's create our masks

In [ ]:
mask_latino = 


In [ ]:
mask_white = 


In [ ]:
mask_black = 


In [ ]:
mask_______ = 


In [ ]:
mask_______ = 


In [ ]:
mask_______ =


Assign the values to a new column `'racen'` for Race/Ethnicity

In [ ]:
state_households.loc[mask_latino, 'racen'] = 'Latino'
state_households.loc[mask_white, 'racen'] = 'White'
state_households.loc[mask_black, 'racen'] = 'Black/African-American'
state_households.loc[mask_______, 'racen'] = '_______'
state_households.loc[mask_______, 'racen'] = '_______'
state_households.loc[mask_______, 'racen'] = '_______'


Checking your results.

Under your new logic, all `race` values should fit into `racen` values so there should not be any null values, right?

Pandas `.isna()` returns a series of either True or False for each value of a series depending on whether or not it is Null. 

AND

in python, True = 1 and False = 0. 

What do you think would happen if you as for the `.sum()` total of a `pandas.Series` of booleans?

***

##### Multiple ways of grouping data

Now that you have derived a working variable for race/ethnicity you can aggregate your data to answer **RQ2**. In pandas, there are many ways to do this, some of them are:
1. `.groupby()` like we've done so far.
2. `.pivot_table()`
3. `pd.crosstabs()` <- this one is a `pandas` method, not a DataFrame method. More later.

##### GroupBy

In [ ]:
state_households.groupby(['racen', '______'])[['______']]._____()

Let's save that to an appropriately named variable since we'll be using it later.

In [ ]:
cihispeed_by_racen = state_households.groupby(['racen', '______'])[['______']]._____()

Now, this grouped dataframe has the total number of households in each of these racen-cihispeed groups. 

We need the share of cihispeed values by racen group. 

In our equation,

$$ \frac{households\ with\ high\ speed\ internet}{total\ households\ in\ racen\ group}$$

We need to find the denominator.

In [ ]:
# find the denominator


In [ ]:
# divide your racen-cihispeed by denominator


In [ ]:
# save to appropriately named variable
shares_cihispeed_by_racen = 

This is a multi-level index dataframe and there are a few ways to slice it. Let's try 3:
1. a classsic `.loc` slice
2. a cross-section (`.xs()`)
3. the `.reset_index()` method

**Classic `.loc`**

In [ ]:
shares_cihispeed_by_racen.loc[INDEX_SLICER, COLUMNS]

**Cross-section**

In [59]:
shares_cihispeed_by_racen.xs?

Signature: shares_cihispeed_by_racen.xs(key, axis=0, level=None, drop_level=True)
Docstring:
Return cross-section from the Series/DataFrame.

This method takes a `key` argument to select data at a particular
level of a MultiIndex.

Parameters
----------
key : label or tuple of label
    Label contained in the index, or partially in a MultiIndex.
axis : {0 or 'index', 1 or 'columns'}, default 0
    Axis to retrieve cross-section on.
level : object, defaults to first n levels (n=1 or len(key))
    In case of a key partially contained in a MultiIndex, indicate
    which levels are used. Levels can be referred by label or position.
drop_level : bool, default True
    If False, returns object with same levels as self.

Returns
-------
Series or DataFrame
    Cross-section from the original Series or DataFrame
    corresponding to the selected index levels.

See Also
--------
DataFrame.loc : Access a group of rows and columns
    by label(s) or a boolean array.
DataFrame.iloc : Purely intege

In [ ]:
shares_cihispeed_by_racen.xs(key = '________', level = _)

**`.reset_index()`**

Another way to slice a multi-level index dataframe is to make it a not-multi-level index dataframe. To do that you need to _reset_ its index. After that, we can slice it how we've been slicing our dataframes previously.

In [ ]:
__________ = ____________._________()

In [ ]:
__________

In [ ]:
mask_yes_cihispeed = (_____________ = '___________')
_______[mask_yes_cihispeed]

***

##### Pivot Tables

The second method of aggregating our data is `.pivot_table()`s.

If you've worked with Excel, you might already be familiar with what a pivot table is.

From [Wikipedia](https://en.wikipedia.org/wiki/Pivot_table):
>A pivot table is a table of statistics that summarizes the data of a more extensive table (such as from a database, spreadsheet, or business intelligence program). This summary might include sums, averages, or other statistics, which the pivot table groups together in a meaningful way.

In [66]:
state_households.pivot_table?

Signature:
state_households.pivot_table(
    values=None,
    index=None,
    columns=None,
    aggfunc='mean',
    fill_value=None,
    margins=False,
    dropna=True,
    margins_name='All',
)
Docstring:
Create a spreadsheet-style pivot table as a DataFrame. The levels in
the pivot table will be stored in MultiIndex objects (hierarchical
indexes) on the index and columns of the result DataFrame.

Parameters
----------
values : column to aggregate, optional
index : column, Grouper, array, or list of the previous
    If an array is passed, it must be the same length as the data. The
    list can contain any of the other types (except list).
    Keys to group by on the pivot table index.  If an array is passed,
    it is being used as the same manner as column values.
columns : column, Grouper, array, or list of the previous
    If an array is passed, it must be the same length as the data. The
    list can contain any of the other types (except list).
    Keys to group by on the pivot 

What we need are four things:
1. What variable will become our `index`?
2. What variable will become our `columns`?
3. What variable will become our `values`?
4. How will we aggregate our values?

Pandas is going to grab each unique value in the variables you choose and use those as rows in your `.index` or separate columns in your `.columns`. The `values` variable should be _quantitative_ in this case (but it doesn't have to be, necessarily). `.pivot_table` will by default find the `mean` of your `values` variable for each cell in your new table, in this case we don't care about the `mean`, we want to `sum` up the total number of households.

Try the following:

```python
state_households.pivot_table(
    index = '______',
    columns = '______', 
    values = 'hhwt',
    aggfunc = '___',
    margins = True,
)
```

Save it to an appropriately named variable.

In [ ]:
households_pivot_table = state_households.pivot_table(
    index = '_____',
    columns = '______',
    ______ = '______',
    ______ = '____',
    _______ = True,
)

What do you think the next step should be?